In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
# System
import os
import sys
sys.path.append('/home/helfrech/Tools/Toolbox/utils')

# Maths
import numpy as np

# Plotting
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D

# ML
from soap import extract_species_pair_groups

from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, balanced_accuracy_score, roc_auc_score
from sklearn.linear_model import Ridge
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor
from sklearn.compose import GridSearchCV

# Utilities
import h5py
import json
import itertools
from copy import deepcopy
from tqdm.notebook import tqdm
import project_utils as utils
from tools import load_json, save_json
from tempfile import mkdtemp
from shutil import rmtree

# Import COSMO style toolkit
import cosmoplot.colorbars as cosmocbars
import cosmoplot.utils as cosmoutils
import cosmoplot.style as cosmostyle

cosmostyle.set_style('article')
colorList = cosmostyle.color_cycle

In [ ]:
# TODO: PCovR from skcosmo

In [ ]:
# TODO: clean up the PCovR/KPCovR functions: add option for scaling by norm

In [ ]:
# TODO: custom scorer for PCovR with loss functions

# Load train and test splits

In [3]:
# Load SOAP cutoffs
soap_hyperparameters = load_json('../Processed_Data/soap_hyperparameters.json')   
cutoffs = soap_hyperparameters['interaction_cutoff']

In [4]:
# Load train sets for IZA and Deem
iza_train_idxs = np.loadtxt('../Processed_Data/IZA_230/svm_train.idxs', dtype=int)
iza_sort_idxs = np.argsort(iza_train_idxs)
iza_unsort_idxs = np.argsort(iza_sort_idxs)
deem_train_idxs = np.loadtxt('../Processed_Data/DEEM_330k/svm_train.idxs', dtype=int)

In [5]:
# Load cantons for IZA and Deem
iza_cantons = np.loadtxt('../Raw_Data/IZA_230/cantons.dat', usecols=1, dtype=int)
deem_cantons_2 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_2-class.dat', dtype=int)
deem_cantons_4 = np.loadtxt('../Processed_Data/DEEM_330k/Data/cantons_4-class.dat', dtype=int)

In [6]:
# Build set of "master" canton labels
cantons = {}

cantons[4] = np.concatenate((
    iza_cantons[iza_train_idxs], 
    deem_cantons_4[deem_train_idxs]
))

cantons[2] = np.concatenate((
    np.ones(len(iza_train_idxs), dtype=int),
    deem_cantons_2[deem_train_idxs]
))

# Build set of class weights (by sample) for centering and scaling
class_weights = {n_cantons: utils.balanced_class_weights(cantons[n_cantons]) for n_cantons in (2, 4)}

# Model setup

In [27]:
model_dir = '../Processed_Data/Models'

deem_name = 'DEEM_330k'
iza_name = 'IZA_230'
deem_dir = f'../Processed_Data/{deem_name}/Data'
iza_dir = f'../Processed_Data/{iza_name}/Data'

In [24]:
# TODO: handle n_components for binary and multiclass cases somehow
pcovr_parameters = dict(n_components=2)

mixings = np.linspace(0.0, 1.0, 11)
alphas = np.logspace(-12, -1, 2)
parameter_grid=dict(
    pcovr__regressor__alpha=alphas, 
    pcovr__regressor__mixing=mixings
)

# Linear PCovR

In [26]:
# Linear model setup
n_species = 2
group_names = {'power': ['OO', 'OSi', 'SiSi', 
                         'OO+OSi', 'OO+SiSi', 'OSi+SiSi',
                         'OO+OSi+SiSi'], 
               'radial': ['O', 'Si', 'O+Si']}

In [ ]:
for cutoff in tqdm(cutoffs, desc='Cutoff', leave=True):
    work_dir = f'{model_dir}/{cutoff}/Linear_Models/PCovR'
    
    for spectrum_type in tqdm(('power', 'radial'), desc='Spectrum', leave=False):
        spectrum_name = spectrum_type.capitalize()
        
        # Load SOAPs
        iza_file = f'{iza_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        iza_soaps = utils.load_hdf5(iza_file, indices=iza_train_idxs[iza_sort_idxs])
        iza_soaps = iza_soaps[iza_unsort_idxs]
        
        deem_file = f'{deem_dir}/{cutoff}/soaps_{spectrum_type}_full_avg_nonorm.hdf5'
        deem_soaps = utils.load_hdf5(deem_file, indices=deem_train_idxs)
        
        soaps = np.vstack((iza_soaps, deem_soaps))
        
        n_features = soaps_train.shape[1]
        feature_groups = extract_species_pair_groups(
            n_features, n_species, 
            spectrum_type=spectrum_type,
            combinations=True
        )
        
        for species_pairing, feature_idxs in zip(
            tqdm(group_names[spectrum_type], desc='Species', leave=False),
            feature_groups
        ):
            
            for n_cantons in tqdm((2, 4), desc='Classes', leave=False):
                
                # Prepare inputs and outputs
                df_dir = f'Linear_Models/SVC/{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                
                # Load decision functions
                iza_dfs = np.loadtxt(f'{iza_dir}/{cutoff}/{df_dir}/svc_structure_dfs.dat')
                iza_dfs = iza_dfs[iza_train_idxs]
                
                deem_dfs = np.loadtxt(f'{deem_dir}/{cutoff}/{df_dir}/svc_structure_dfs.dat')
                deem_dfs = deem_dfs[deem_train_idxs]
                
                dfs = np.concatenate((iza_dfs, deem_dfs))
                                
                cache_dir = mkdtemp()
                
                pipeline = Pipeline(
                    [
                        ('norm_scaler', utils.StandardNormScaler()),
                        ('pcovr', TransformedTargetRegressor(
                            regressor=PCovR(**pcovr_parameters),
                            transformer=utils.StandardNormScaler(featurewise=True)
                        ))
                    ],
                    memory=cache_dir
                )
                
                # IZA + DEEM classification
                gscv = GridSearchCV(
                    pipeline, parameter_grid,
                    scoring={
                        regression_loss, # TODO: scoring wrapper or `score` method in the PCovR class
                        projection_loss # TODO: do we also need weighted regression scoring?
                    },
                    cv=StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0),
                    refit=False, return_train_score=True, error_score='raise'
                )
                fit_params = {
                    'norm_scaler__sample_weight': class_weights[n_cantons],
                    'pcovr__regressor__sample_weight': class_weights[n_cantons],
                    'pcovr__transformer__sample_weight': class_weights[n_cantons]
                }
                gscv.fit(soaps[:, feature_idxs], dfs, **fit_params)
                
                # Prepare outputs
                output_dir = f'{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                os.makedirs(f'{work_dir}/{output_dir}', exist_ok=True)
                
                save_json(gscv.cv_results_, f'{work_dir}/{output_dir}/cv_results.json')
                rmtree(cache_dir)

## Check the cross-validated parameters

In [ ]:
# TODO: also extract optimal for mixing = 0.0 and mixing = 1.0

In [ ]:
# IZA + DEEM classification
for cutoff in cutoffs:
    work_dir = f'{model_dir}/{cutoff}/Linear_Models/PCovR'
    for spectrum_type in ('power', 'radial'):
        spectrum_name = spectrum_type.capitalize()
        for group_name in group_names[spectrum_type]:
            for n_cantons in (2, 4):
                result_dir = f'{n_cantons}-Class/{spectrum_name}/{species_pairing}'
                cv_results = load_json(f'){work_dir}/{output_dir}/cv_results.json', array_convert=True)
                print(f'-----Optimal Parameters for {cutoff} {spectrum_type} {group_name} {n_cantons} -----')
                for score in (): # TODO: names of custom scorers
                    idx = np.argmin(cv_results[f'rank_test_{score}'])
                    opt_parameters = utils.get_optimal_parameters(cv_results, score, **svm_parameters)
                    print(f'{score} =', cv_results[f'mean_test_{score}'][idx])
                    print(opt_parameters)
                    print('')
                    
                    save_json(opt_parameters, f'{work_dir}/{output_dir}/pcovr_parameters_{score}.json')

## Check for optimization problems

In [ ]:
for cutoff in cutoffs:
    work_dir = f'{model_dir}/{cutoff}/Linear_Models/PCovR'
    
    for spectrum_type in ('power', 'radial'):
        spectrum_name = spectrum_type.capitalize()
        
        for species_pairing, feature_idxs in zip(group_names[spectrum_type], feature_groups):
                        
            for n_cantons in (2, 4):
                
                print(f'Cutoff: {cutoff}, Spectrum: {spectrum_name}, Species: {species_pairing}, {n_cantons}-Class')
                
                output_dir = f'{n_cantons}-Class/{spectrum_name}/{species_pairing}'
            
                loss_file = f'{work_dir}/{output_dir}/pcovr_optimization.dat'

                # TODO: Load losses
                

                fig = plt.figure(figsize=(7.0, 3.5))
                axs_loss_sum_train = fig.add_subplot(1, 2, 1)
                axs_loss_sum_test = fig.add_subplot(1, 2, 2)

                # Sum of projection and regression loss over all cantons for the train set
                axs_loss_sum_train.semilogy(alphas, lr_train_matrix[:, opt_reg_idx], 'o-', label='l_regr')
                axs_loss_sum_train.semilogy(alphas, lp_train_matrix[:, opt_reg_idx], 'o-', label='l_proj')
                axs_loss_sum_train.semilogy(alphas, lr_train_matrix[:, opt_reg_idx]
                                            + lp_train_matrix[:, opt_reg_idx], 'o-', label='l_regr+l_proj')

                axs_loss_sum_train.legend()
                axs_loss_sum_train.set_title('Train')
                axs_loss_sum_train.set_xlabel('alpha')
                axs_loss_sum_train.set_ylabel('loss')

                # Sum of projection and regression loss over all cantons for the test set
                axs_loss_sum_test.semilogy(alphas, lr_test_matrix[:, opt_reg_idx], 'o-', label='l_regr')
                axs_loss_sum_test.semilogy(alphas, lp_test_matrix[:, opt_reg_idx], 'o-', label='l_proj')
                axs_loss_sum_test.semilogy(alphas, lr_test_matrix[:, opt_reg_idx]
                                           + lp_test_matrix[:, opt_reg_idx], 'o-', label='l_regr+l_proj')

                axs_loss_sum_test.legend()
                axs_loss_sum_test.set_title('Test')
                axs_loss_sum_test.set_xlabel('alpha')
                axs_loss_sum_test.set_ylabel('loss')

                plt.show()